<a href="https://colab.research.google.com/github/Ankur3107/GitHub-Bugs-Prediction-Challenge/blob/main/roberta-xlm-base-newsampling-maxlen320-2epochs-2lac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.1MB 3.4MB/s 
     |████████████████████████████████| 890kB 17.7MB/s 
     |████████████████████████████████| 1.1MB 24.8MB/s 
     |████████████████████████████████| 3.0MB 36.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=dfc9387bc4b9c74782227f9313f007d957b54a734646754931717eeeb29b1a5c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!wget https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
!unzip ./Embold_Participant\'s_Dataset.zip -d Dataset

--2020-10-12 12:35:33--  https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 52.219.66.120
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|52.219.66.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102320961 (98M) [application/octet-stream]
Saving to: ‘Embold_Participant's_Dataset.zip’

Embold_Participant' 100%[===================>]  97.58M  10.8MB/s    in 10s     

2020-10-12 12:35:44 (9.65 MB/s) - ‘Embold_Participant's_Dataset.zip’ saved [102320961/102320961]

Archive:  ./Embold_Participant's_Dataset.zip
   creating: Dataset/Embold_Participant's_Dataset/
  inflating: Dataset/Embold_Participant's_Dataset/sample submission.csv  
  inflating: Dataset/__MACOSX/Embold_Participant's_Dataset/._sample submission.csv  
  inflating: Dataset/Embold_Participant's_Dataset/embold_train_ext

In [3]:
cd "Dataset/Embold_Participant's_Dataset/"

/content/Dataset/Embold_Participant's_Dataset


In [4]:
#import pandas as pd
import numpy as np
import pandas as pd

In [5]:
train_df = pd.read_json("embold_train.json").reset_index(drop=True)
test_df = pd.read_json("embold_test.json").reset_index(drop=True)
train_ex_df = pd.read_json("embold_train_extra.json")

In [6]:
train_data = train_df.append(train_ex_df)
test_df['text'] = test_df['title']+' '+test_df['body']
train_data['text'] = train_data['title']+' '+train_data['body']

In [7]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import *
from transformers import pipeline
from pprint import pprint
from tensorflow import keras

In [8]:
@dataclass
class Config:
    MAX_LEN = 320
    BATCH_SIZE = 16  # per TPU core
    TOTAL_STEPS = 2000  # thats approx 4 epochs
    EVALUATE_EVERY = 200
    LR = 1e-5
    PRETRAINED_MODEL = "xlm-roberta-base"  # huggingface bert model


transformer_flags = Config()


def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print("Running on TPU ", tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = transformer_flags.BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.82.47.58:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.82.47.58:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.82.47.58:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [16]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_mask=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True,
    )

    return np.array(enc_di["input_ids"])

from transformers import *
#tokenizer = AutoTokenizer.from_pretrained(transformer_flags.PRETRAINED_MODEL)
#tokenizer = BertTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)
#tokenizer = AlbertTokenizer.from_pretrained(transformer_flags.PRETRAINED_MODEL, use_fast=True)
tokenizer = AutoTokenizer.from_pretrained(transformer_flags.PRETRAINED_MODEL, use_fast=True)
#tokenizer = RobertaTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)
#tokenizer = ElectraTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)

OSError: ignored

In [14]:
def build_model():
  input_layer = tf.keras.layers.Input((transformer_flags.MAX_LEN,), dtype=tf.int32)
  bert_model = TFAutoModel.from_pretrained(transformer_flags.PRETRAINED_MODEL, from_pt=True)
  #bert_model = TFAutoModel.from_pretrained('/content/github-albert-xlarge-v2/')
  #bert_model = TFAutoModel.from_pretrained('../../github-roberta-large/')
  output_layer = bert_model(input_layer) 
  '''x1 = tf.keras.layers.Dropout(0.1)(output_layer[0])
  x1 = tf.keras.layers.Conv1D(768, 2,padding='same')(x1)
  x1 = tf.keras.layers.LeakyReLU()(x1)
  x1 = tf.keras.layers.Conv1D(64, 2,padding='same')(x1)
  x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)'''
  #pooled_output = tf.keras.layers.GlobalAveragePooling1D()(output_layer[1])
  output = tf.keras.layers.Dense(3, activation='softmax')(output_layer[1])
  classifier_model = tf.keras.Model(input_layer, output)

  optimizer = tf.keras.optimizers.Adam(learning_rate=transformer_flags.LR)
  classifier_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  classifier_model.summary()
    
  return classifier_model

In [12]:
df = train_data[train_data.label==2]
df = df.append(train_data[train_data.label!=2].sample(200000, random_state=3107))
train, validate = np.split(df.sample(frac=1, random_state=3107), [int(.8*len(df))])
train.shape, validate.shape

((193760, 4), (48441, 4))

In [15]:
with strategy.scope():
    classifier_model = build_model()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 320)]             0         
_________________________________________________________________
tfxlm_roberta_model (TFXLMRo ((None, 320, 768), (None, 278043648 
_________________________________________________________________
dense (Dense)                (None, 3)                 2307      
Total params: 278,045,955
Trainable params: 278,045,955
Non-trainable params: 0
_________________________________________________________________


In [19]:
# data preparation
X_data = regular_encode(train.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)
y_train = train.label.values
print(X_data.shape, y_train.shape)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(193760, 320) (193760,)


In [20]:
# data preparation
X_val = regular_encode(validate.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)
y_val = validate.label.values
print(X_val.shape, y_val.shape)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(48441, 320) (48441,)


In [21]:
X_test = regular_encode(test_df.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [22]:
classifier_model.fit(X_data, y_train, epochs=2,batch_size=64, validation_data=(X_val, y_val))

Epoch 1/2
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:429: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 192001536 elements. This may consume a large amount of memory.
  num_elements)


   2/3028 [..............................] - ETA: 16:03:15 - loss: 1.2051 - accuracy: 0.1875WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0111s vs `on_train_batch_end` time: 0.2247s). Check your callbacks.


3028/3028 [==============================] - ETA: 0s - loss: 0.5958 - accuracy: 0.7579WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0037s vs `on_test_batch_end` time: 0.0567s). Check your callbacks.


3028/3028 [==============================] - 838s 277ms/step - loss: 0.5958 - accuracy: 0.7579 - val_loss: 0.4988 - val_accuracy: 0.8109
Epoch 2/2
3028/3028 [==============================] - 757s 250ms/step - loss: 0.4850 - accuracy: 0.8161 - val_loss: 0.4864 - val_accuracy: 0.8135


In [23]:
y_val_pred = classifier_model.predict(X_val, batch_size=global_batch_size, verbose=1)
y_test_pred = classifier_model.predict(X_test, batch_size=global_batch_size, verbose=1)

  2/379 [..............................] - ETA: 23:13WARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0034s vs `on_predict_batch_end` time: 0.0883s). Check your callbacks.


  2/235 [..............................] - ETA: 10sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0037s vs `on_predict_batch_end` time: 0.0850s). Check your callbacks.


235/235 [==============================] - 27s 113ms/step


In [24]:
eval_df = pd.DataFrame({'X':y_val_pred.tolist(), 'y':y_val.tolist()})

In [25]:
eval_df.to_csv('roberta-xlm-base-newsampling-maxlen320-2epochs-2lac-eval-prob.csv', index=False)

In [26]:
submission = pd.read_csv('sample submission.csv')
submission['label'] = y_test_pred.tolist()
file_name = 'roberta-xlm-base-newsampling-maxlen320-2epochs-2lac-test-prob.csv'
submission.to_csv(file_name, index=False)